# Deep Learning & Applied AI

# Tutorial 2: Tensor operations

In this tutorial, we will cover:

- Tensor operations: broadcasting, (not)-elementwise operations, tensor contraction, einsum

Our info:

- Based on original material by Dr. Antonio Norelli (norelli@di.uniroma1.it)

Course:

- Website and notebooks will be available at https://erodola.github.io/DLAI-s2-2024/



## Introduction

In this tutorial we will continue to learn basic tensor usage, we will cover  broadcasting, fundamental linear algebra operations, and finally `einsum`, a single operation implementing the Einstein notation to rule them all!

All these tensor operations will come in handy to build our deep neural networks.
Yet, the high level API offered by PyTorch to perform GPU-acccelerated linear algebra operations may turn useful in many other fields, from microbiology to fluid dynamics.

The GPU computing paradigm offers several benefits over single-core machines or traditional supercomputers equipped with many single-core nodes.
Deep learning frameworks such as the one we are studying are a very good compromise between simplicity and expressivenes to unleash the power of GPU-computing.

To get even more control you can tackle directly the CUDA language, but we won't go there with this course!

## PyTorch

**Reminder:** Familiarize with the [PyTorch Documentation](https://pytorch.org/docs/stable/) as it will greatly assist you.






In [ ]:
import torch
import numpy as np

In [ ]:
# Utility print function

from typing import Union

def print_arr(*arr: Union[torch.Tensor, np.ndarray], prefix: str = "") -> None:
    """Pretty print tensors, together with their shape and type

    :param arr: one or more tensors
    :param prefix: prefix to use when printing the tensors
    """
    print(
        "\n\n".join(
            f"{prefix}{str(x)} <shape: {x.shape}> <dtype: {x.dtype}>" for x in arr
        )
    )

####Set torch and numpy random seeds for reproducibility

As we will see, several operations in deep learning (e.g. training a network) rely on randomness in order to work effectively. This means that we will get different results each time we run a test, which can make design and debugging difficult.

To this end, we usually **set a fixed seed** for the pseudo-random number generator, so that we are sure to always see the "same randomness" that makes our tests reproducible.

> Once your model works, remember to test multiple times _without_ a fixed seed! The results you got at design time may be due to overfitting the seed (e.g. you have chosen hyperparameters that happen to work particularly well with a given seed.), or just out of luck.

If you are going to use a gpu, two further options must be set.

In [ ]:
import random

np.random.seed(42)
random.seed(0)

torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True  # Note that this Deterministic mode can have a performance impact
torch.backends.cudnn.benchmark = False

# We will see frameworks that aid the reproducibility of your code,
# e.g. PyTorch Lightning exposes a `seed_everything` function by default:
# https://github.com/PyTorchLightning/pytorch-lightning/blob/e1f5eacab98670bc1de72c88657404a15aadd527/pytorch_lightning/utilities/seed.py#L29

### **Tensor operations**



In [ ]:
t = torch.rand(3,3)
t

Functions that operate on tensors are often accessible in different ways:

- From the **`torch` module**...:

In [ ]:
torch.add(t, t)

- ...or by tensors **methods**:

In [ ]:
t.add(t)

- ...or even through **overloaded** operators:




In [ ]:
t + t

None of the above operates in-place:

In [ ]:
# t is unchanged
t

These functions are all equivalent, they are *aliases* of the same method.
Personal preference, code consistency, and readability should guide your decision of which one to use.

> e.g. `torch.add(...)` may be too verbose, but in some cases it may be preferable since it makes explicit to the code-reader that you are dealing with tensors. Nevertheless, if you are using [types](https://docs.python.org/3/library/typing.html) -- and you should be using types -- it will be rarely necessary.


Most operation in PyTorch are **not in-place**. It means that the resulting tensor is a *new* tensor, and it does not share the underlying data with other tensors. Changes to the new tensor are not reflected to other tensors.
We will see in future tutorials why this is important, for now a TLDR is: **in-place operations may break the auto-differentiation mechanism**.


In-place operations are still available in PyTorch, and in some cases (e.g. when you don't need autodiff) they can be useful; they are more efficient, since they never require to perform deep copies of the data.
They are normally recognized by a trailing `_`:

In [ ]:
t

In [ ]:
t.add_(t)  # notice the trailing _

In [ ]:
t  # t itself changed!

Another common in-place operation is the assignment:

In [ ]:
t[0] = 42
t

#### **Basic operations and broadcasting**

Basic mathematical operations $(+, -, *, /, **)$ are applied **element-wise**: for example, if `x` and `y` are two tensors, the product `x*y` is a tensor with the same size, and its values are the element-wise products of the two tensors. In mathematics, this is also called a Hadamard product.

**Broadcasting** is another powerful mechanism that allows PyTorch to perform operations on tensors of different shapes. The most basic example is summing a scalar (a rank-0 tensor) to a matrix (a rank-2 tensor).

In [ ]:
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float64)
y = torch.tensor([[5, 6], [7, 8]], dtype=torch.float64)

print(x + y)  # element-wise sum
print(x + 4.2)  # broadcasting

In [ ]:
# other examples
print(x * y - 5)
print((x - y) / y)  # element-wise division!

Broadcasting is quite powerful! When you perform an operation between two tensors with different shape, PyTorch automatically "broadcasts" the smaller tensor across the larger tensor so that they have compatible shapes.

In the example below, the sequence `v` is replicated (_without actually copying data!_) along the missing dimension so that it fits the shape of matrix `m`:

In [ ]:
m = torch.arange(12).reshape(4, 3)
v = torch.tensor([100, 0, 200])
n = m + v
print_arr(m, v, n)

In this other example `m` and `u` are both rank-2, but the smaller one (`u`) is expanded along the dimension where it has size 1 to fit `m`:

In [ ]:
m = torch.arange(12).reshape(4, 3)
u = torch.tensor([0, 10, 0, 20]).reshape(4,1)
n = m + u
print_arr(m, u, n)

In the following example, both tensors are expanded along their size-1 dimensions, so that the sum makes sense:

In [ ]:
w = u + v
print_arr(u, v, w)

Mastering broadcasting is hard!

However, it is very convenient as it allows writing **vectorized** code, i.e., code that avoids explicit python loops which can not be efficiently parallelized.

Technically, broadcasting takes advantage of the underlying C implementation of PyTorch and Numpy (on CPU) or CUDA implementation of Pytorch (on GPU). Here's a take-home illustration for your convenience:

![broadcasting](https://jakevdp.github.io/PythonDataScienceHandbook/figures/02.05-broadcasting.png)

##### **EXERCISE**
>
> Given two vectors $x \in \mathbb{R}^n$ and $y \in \mathbb{R}^m$, compute the differences between all possible pairs of their elements, and organize these differences in a matrix $Z \in \mathbb{R}^{n \times m}$:
> $$ z_{ij} = x_i - y_j $$

In [ ]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5])

# ✏️ your code here

In [ ]:
# @title 👀 Solution


out = x.unsqueeze(1) - y

# equivalent to:
# x.reshape([-1, 1]) - y

# equivalent to:
# x[:, None] - y

# equivalent to:
# x[:, None] - y[None, :]

print_arr(x, y, out)

#### 📖 **Broadcasting, let's take a peek under the hood**

To recap: if a PyTorch operation supports broadcast, then **its tensor arguments can be implicitly expanded to be of equal sizes** (without making copies of the data).

###### **Broadcastable tensors**

Two tensors are "broadcastable" if:
- Each tensor has at least one dimension
- When iterating over the dimension sizes, starting at the trailing dimension, the dimension **sizes** must either **be equal**, **one of them is 1**, or **one of them does not exist**.


###### **Broadcasting rules**

Broadcasting two tensors together follows these rules:

1. If the input tensors have different ranks, **singleton dimensions are prepended to the shape** of the smaller one until it has the same rank as the other
2. The size in each dimension of the **output shape** is the maximum size in that dimension between the two tensors
3. An input can be used in the computation if its size in a particular **dimension either matches** the output size in that dimension, **or is a singleton dimension**
4. If an input has a dimension size of 1 in its shape, the **first data entry in that dimension will be used for all calculations** along that dimension.

**Example**:

- `m` has shape `[4, 3]`
- `v` has shape `[3,]`.


In [ ]:
print_arr(m, v)

In [ ]:
n = m + v
print_arr(n)


Following the Broadcasting logic, this is what happened:

- `v` has less dims than `m` so a dimension of `1` is **prepended** $\to$ `v` is now `[1, 3]`.
- Output shape will be `[max(1, 4), max(3, 3)] = [4, 3]`.
- `dim 1` of `v` matches exactly `3`; `dim 0` is `1`, so we can use the first data entry in that dimension (i.e. the whole `row 0` of `v`) each time any row is accessed. This is effectively like converting `v` from `[1, 3]` to `[4, 3]` by stacking the repeated row four times.


For more on broadcasting, see the [documentation](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html).

Functions that support broadcasting are known as universal functions (i.e. ufuncs). For Numpy you can find the list of all universal functions in the [documentation](https://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs).

#### **EXERCISE**
>
> Given a tensor $Y \in \mathbb{R}^{n \times m}$ and an index pair $(a,b)$, for each element of $Y$ compute its $L_p$ distance to $(a,b)$, and store the resulting distance value in the corresponding cell of $Y$.
>
> In brief, compute:
> $$ y_{ij} = d_{L_p}\left( (i,j), (a,b) \right) \text{ for all }  i,j$$
>
> and visualize the resulting $Y$.
>
> Try different values of $p>0$ to see what happens.
>
> ---
>
> The [$L_p$ distance](https://en.wikipedia.org/wiki/Lp_space#The_p-norm_in_finite_dimensions) between two points $x$ and $y$ can be computed as: $d_{L_p}(x, y)=\left( \sum_{i=1}^n|x_i - y_i|^p\right)^{1/p}$
>
> Example: The $L_1$ distance between $(i,j) = (3, 5)$ and $(a,b) = (14, 20)$ is:
> $$ y_{3,5} = d_{L_1}( (3, 5), (14, 20) ) = |3 - 14| + |5 - 20| $$

In [ ]:
# @title Utility function, you can execute and safely ignore this cell

import plotly.express as px

def plot_row_images(images: Union[torch.Tensor, np.ndarray]) -> None:
  """ Plots the images in a subplot with multiple rows.

  Handles correctly grayscale images.

  :param images: tensor with shape [number of images, width, height, <colors>]
  """
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  fig = make_subplots(rows=1, cols=images.shape[0] ,
                      specs=[[{}] * images.shape[0]])

  # Convert grayscale image to something that go.Image likes
  if images.dim() == 3:
    images = torch.stack((images, images, images), dim= -1)
  elif (images.dim() == 4 and images.shape[-1] == 1):
    images = torch.cat((images, images, images), dim= -1)

  assert images.shape[-1] == 3 or images.shape[-1] == 4

  for i in range(images.shape[0]):
    i_image = np.asarray(images[i, ...])

    fig.add_trace(
        go.Image(z = i_image, zmin=[0, 0, 0, 0], zmax=[1, 1, 1, 1]),
        row=1, col=i + 1
    )

  fig.show()


# When using plotly pay attention that it often does not like PyTorch Tensors
# ...and it does not give any error, just a empty plot.

In [ ]:
x = torch.zeros(300, 300)
a = 150
b = 150

x[a, b] = 1  # this will be overwritten by your distance-calculating code
plot_row_images(x[None, :])

In [ ]:
# ✏️ your code here

In [ ]:
# @title 👀 Solution


rows = torch.arange(x.shape[0])
cols = torch.arange(x.shape[1])

# Manual computation of L1
y = (torch.abs(rows - a)[:, None] + torch.abs(cols - b)[None, :])
px.imshow(y).show()

In [ ]:
# @title 👀 Solution


# Parametric computation of Lp
p = 8
y = ((torch.abs(rows - a ) ** p )[:, None] +
     (torch.abs(cols - b) ** p)[None, :]) ** (1/p)
px.imshow(y).show()

Try Solution 2 with `p=10`. What happens, and why?

In [ ]:
# @title 👀 Solution


# This works even with p=10. Why?
p = 10
y = ((torch.abs(rows.double() - a ) ** p )[:, None] +
     (torch.abs(cols.double() - b) ** p)[None, :]) ** (1/p)
px.imshow(y).show()

# -> Write your own explanation here
p = 10
print(torch.tensor(10, dtype=torch.int) ** p)
print(torch.tensor(10, dtype=torch.double) ** p)

#### **Non-elementwise operations**


PyTorch and NumPy provide many useful functions to perform computations on tensors:

In [ ]:
x = torch.tensor([[1, 2, 3], [3, 4, 5]], dtype=torch.float32)
print_arr(x)

In [ ]:
# Sum up all the elements
print_arr(torch.sum(x))

In [ ]:
# Compute the mean of each column
print_arr(torch.mean(x, dim=0))

> **REMEMBER!**
>
> In order to avoid confusion with the `dim` parameter, you can think of it as an **index over the list returned by `tensor.shape`**. The operation is performed by iterating over that dimension.
>
> Example above: since our tensor `x` has shape `[2, 3]`, the dimension `dim=0` operates along the `2`.
>
> Visually (here array means _tensor_):
>
><img src="https://qph.fs.quoracdn.net/main-qimg-30be20ab9458b5865b526d287b4fef9a" width="500" >

In [ ]:
print_arr(x)

In [ ]:
# Compute the product of each row
print_arr(torch.prod(x, dim=1))

In [ ]:
# Max along the rows (i.e. max value in each column)
values, indices = torch.max(x, dim=0)
print_arr(values)

In [ ]:
# Max along the columns (i.e. max value in each row)
values, indices = torch.max(x, dim=1)
print_arr(values)

##### 📖 **Dim parameter, let's take a peek under the hood**


Let's see what the `dim` parameter exactly does:

In [ ]:
dim = 2

a = torch.arange(2*3*4).reshape(2, 3, 4)
out = a.sum(dim=dim)
out

In [ ]:
# It is summing over the `dim` dimension, i.e.:
a.shape

In [ ]:
# The `dim` dimension has 4 elements
a.shape[dim]

In [ ]:
# The dimension dim collapses, the output tensor will have shape:
new_shape = a.shape[:dim] + a.shape[dim + 1:]
new_shape

In [ ]:
# Explicitly compute the sum over dim
out = torch.zeros(new_shape)

# iterate over all the rows
for r in range(a.shape[0]):
  # iterate over all the columns in the r-th row
  for c in range(a.shape[1]):

    for i in range(a.shape[dim]): # <- sum over 'dim'

      out[r, c] += a[r, c, i]

out

# **DO NOT** use for loops in your code

###### **EXERCISE**
>
> Given a matrix $X \in R^{k \times k}$:
> - Compute the mean of the values along its diagonal.
>
> Perform this computation in at least two different ways, then check that the results are the same.

In [ ]:
x = torch.rand(4, 4)
print_arr(x)

In [ ]:
# ✏️ your code here

In [ ]:
# @title 👀 Solution


a = torch.mean(x[torch.arange(x.shape[0]), torch.arange(x.shape[1])])
b = torch.sum(torch.eye(x.shape[0]) * x) / x.shape[0]
c = torch.trace(x) / x.shape[0]
d = torch.mean(torch.diag(x))

print(torch.equal(a, b) and torch.equal(a, c) and torch.equal(a, d))
print_arr(a)

##### **EXERCISE**
>
> Given a binary non-symmetric matrix $X \in \{0, 1\}^{n\times n}$, build the symmetric matrix $Y \in \{0, 1\}^{n \times n}$ defined as:
> $$
y_{ij} =
\begin{cases}
1 & \text{if } x_{ij} = 1 \\
1 & \text{if } x_{ji} = 1 \\
0 & \text{otherwise}
\end{cases}
$$
>
> *Hint*: search for `clamp` in the [docs](https://pytorch.org/docs/stable/index.html)

In [ ]:
x = torch.randint(0, 2, (5, 5))  # Non-symmetric matrix
x

In [ ]:
# ✏️ your code here

In [ ]:
# @title 👀 Solution


(x + x.transpose(1, 0)).clamp(max=1)

#### **Tensor contractions**

##### **Matrix multiplication**

Given $X \in R^{n \times d}$ and $Y \in R^{d \times v}$, their matrix multiplication $Z \in R^{n \times v}$ is defined as:

$$ \sum_{k=0}^{d} x_{ik} y_{kj} = z_{ij} $$


In [ ]:
x = torch.tensor([[1, 2], [3, 4], [5, 6]])
y = torch.tensor([[1, 2], [2, 1]])
print_arr(x, y)

In [ ]:
# as we will see, matmul's functionality is not limited to matrix-matrix multiplication
torch.matmul(x, y)

In [ ]:
x @ y  # Operator overload for matmul

In [ ]:
torch.mm(x, y)  # PyTorch function, only works for rank-2 tensors (matrices) https://pytorch.org/docs/stable/generated/torch.mm.html

In [ ]:
x.mm(y)  # Tensor method

In [ ]:
torch.einsum('ik, kj -> ij', (x, y))  # Einsum notation!

# It summed up dimension labeled with the index `k`

##### **Dot product**
Also known as scalar product or inner product.
Given $x \in \mathbb{R}^k$ and $y \in \mathbb{R}^k$, the dot product $z \in \mathbb{R}$ is defined as:

$$ \sum_{i=0}^{k} x_i y_i = z $$

In [ ]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])
print_arr(x, y)

In [ ]:
# We want to perform:
(1 * 4) + (2 * 5) + (3 * 6)

In [ ]:
torch.dot(x, y)  # PyTorch function

In [ ]:
x.dot(y) # Tensor method

In [ ]:
x @ y  # PyTorch operator again overloading matmul

In [ ]:
torch.einsum('i, i ->', (x, y))  # Einstein notation!

# Read it as:
# - iterate with i along x
# - iterate with i along y
# - compute the product at each iteration
# - sum the products and return a scalar (-> means return a scalar)

# More in general, Einstein notation:
# Multiply point-wise repeated indices in the input
# Sum up along the indices that `do not` appear in the output

# More on this below!

##### **Batch matrix multiplication**

Often we want to perform more operations together. Why?
- Reduce the **overhead of uploading** each tensor to/from the GPU memory
- **Better parallelization** of the computation

Given two 3D tensors, each one containing ``b`` matrices,
$X \in \mathbb{R}^{b \times n \times m}$
and  
$Y \in \mathbb{R}^{b \times m \times p}$,

We want to multiply together each $i$-th pair of matrices, obtaining a tensor $Z \in \mathbb{R}^{b \times n \times p}$ defined as:

$$ z_{bij} = \sum_{k=0}^m x_{bik} y_{bkj} $$

In [ ]:
# here b = 2 matrices
x = torch.tensor([[[1, 2], [3, 4], [5, 6]], [[1, 2], [3, 4], [5, 6]]])  # 3x2 matrices
y = torch.tensor([[[1, 2], [2, 1]], [[1, 2], [2, 1]]])  # 2x2 matrices
print_arr(x, y)

In [ ]:
torch.bmm(x, y)  # **not** torch.mm

In [ ]:
# Operator overload! again, matmul is actually doing the job
x @ y

In [ ]:
torch.einsum('bik, bkj -> bij', (x, y)) # Einstein notation!

##### 📖 **Broadcast matrix multiplication**

Given a matrix $Y \in \mathbb{R}^{m \times p}$ and $b$ matrices of size $n \times m$ organized in a 3D tensor $X \in \mathbb{R}^{b \times n \times m}$, we want to multiply together each matrix $X_{i,:,:}$ with $Y$, obtaining a tensor $Z \in R^{b \times n \times p}$ defined as:

$$ z_{bij} = \sum_{k=0}^m x_{bik} y_{kj} $$


In [ ]:
x = torch.tensor([[[1, 2], [3, 4], [5, 6]], [[1, 2], [3, 4], [5, 6]]])
y = torch.tensor([[1, 2], [2, 1]])
print_arr(x, y)

In [ ]:
torch.matmul(x, y)  # always uses the last two dimensions

In [ ]:
x @ y   # still using the last two dimensions since @ overloads matmul

##### **EXERCISE**
>
> Use the einsum notation to compute the equivalent broadcast matrix multiplication!

### **Einsum notation**

Einstein notation is a way to express complex operations on tensors.

- It is **concise but expressive enough** to perform almost every operation you will need in building your neural networks, allowing you to think of the only thing that matters... **dimensions!**
- You will **not need to check your dimensions** after an einsum operation, since the dimensions themselves are *defining* the tensor operation.
- You will **not need to shape-comment** your tensors. Those comments do not work: they are bound to get outdated.
-  You will not need to explicitly code **intermediate operations** such as reshaping, transposing and intermediate tensors.
- It is **not library-specific**, being avaiable in ``numpy``, ``pytorch``, ``tensorflow`` and ``jax`` with the same signature. So you do not need to remember the functions signature in all the frameworks.
- It can sometimes be compiled to high-performing code (e.g. [Tensor Comprehensions](https://pytorch.org/blog/tensor-comprehensions/))

Check [this blog post by Olexa Bilaniuk](https://obilaniu6266h16.wordpress.com/2016/02/04/einstein-summation-in-numpy/) to take a peek under the hood of einsum and [this one by Tim Rocktäschel](https://rockt.github.io/2018/04/30/einsum) for several examples.

Its formal behavior is well described in the [Numpy documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html).
However, it is very intuitive and better explained through examples.

![alt text](https://obilaniu6266h16.files.wordpress.com/2016/02/einsum-fmtstring.png?w=676)

> *Historical note (taken from O.Bilaniuk's post)*
>
> Einstein had no part in the development of this notation. He merely popularized it, by expressing his entire theory of General Relativity in it. In a letter to [Tullio Levi-Civita](https://en.wikipedia.org/wiki/Tullio_Levi-Civita), co-developer alongside [Gregorio Ricci-Curbastro](https://en.wikipedia.org/wiki/Gregorio_Ricci-Curbastro) of Ricci calculus (of which this summation notation was only a part), Einstein wrote:
>
> " *I admire the elegance of your method of computation; it must be nice to ride through these fields upon the horse of true mathematics while the like of us have to make our way laboriously on foot.* "

In [ ]:
a = torch.arange(6).reshape(2, 3)  # will use this in the examples below

###### **Matrix transpose**

$$ B_{ji} = A_{ij} $$

In [ ]:
# The characters are indices along each dimension
b = torch.einsum('ij -> ji', a)
print_arr(a, b)

###### **Sum**

$$ b = \sum_i \sum_j A_{ij} := A_{ij} $$


In [ ]:
# Indices that do not appear in the output tensor are summed up
b = torch.einsum('ij -> ', a)
print_arr(a, b)

###### **Column sum**

$$ b_j = \sum_i A_{ij} := A_{ij} $$

In [ ]:
# Indices that do not appear in the output tensor are summed up,
# ...even if some other index appears
b = torch.einsum('ij -> j', a)
print_arr(a, b)

###### **EXERCISE**
>
> Given a binary tensor $X \in \{0, 1\}^{n \times m}$ return a tensor $y \in \mathbb{R}^{n}$ that has in the $i$-th position the **number of ones** in the $i$-th row of $X$.
>
>Give a solution using `einsum`, and a solution using standard manipulation.

In [ ]:
x = (torch.rand(100, 200) > 0.5).int()

In [ ]:
# Display a binary matrix with plotly

fig = px.imshow(x)
fig.show()

In [ ]:
# ✏️ your code here


In [ ]:
# @title 👀 Solution


# Count the number of ones in each row
row_ones = torch.einsum('ij -> i', x)

row_ones2 = torch.sum(x, dim=-1)  # recall that -1 refers to the last dimension

torch.equal(row_ones, row_ones2)

In [ ]:
px.imshow(row_ones[:, None]).show()
print(f'Sum up the row counts: {row_ones.sum()}\nSum directly all the ones in the matrix: {x.sum()}')

###### **Matrix-vector multiplication**

$$ c_i = \sum_k A_{ik}b_k := A_{ik}b_k $$

In [ ]:
# Repeated indices in different input tensors indicate pointwise multiplication
a = torch.arange(6).reshape(2, 3)
b = torch.arange(3)
c = torch.einsum('ik, k -> i', [a, b])  # Multiply on k, then sum up on k
print_arr(a, b, c)

###### **Matrix-matrix multiplication**

$$ C_{ij} = \sum_k A_{ik}B_{kj} := A_{ik}B_{kj} $$

📖 Understanding einsum, what happens inside?

![alt text](https://obilaniu6266h16.files.wordpress.com/2016/02/einsum-matrixmul.png?w=676)

In [ ]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(15).reshape(3, 5)
c = torch.einsum('ik, kj -> ij', [a, b])
print_arr(a, b, c)

###### **Dot product multiplication**

$$ c = \sum_i a_i b_i := a_i b_i $$

In [ ]:
a = torch.arange(3)
b = torch.arange(3,6)
c = torch.einsum('i,i->', (a, b))
print_arr(a, b, c)

###### **Point-wise multiplication**
Also known as Hadamard product:

$$ C_{ij} = A_{ij} B_{ij} $$

In [ ]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(6,12).reshape(2, 3)
c = torch.einsum('ij, ij -> ij', (a, b))
print_arr(a, b, c)

###### **Outer product**
Given two column vectors of length $m$ and $n$ respectively,
\begin{align*}
\mathbf{a}=\left[\begin{array}{c}
a_{1} &
a_{2} &
\dots &
a_{m}
\end{array}\right]^\top, \quad \mathbf{b}=\left[\begin{array}{c}
b_{1} &
b_{2} &
\dots &
b_{n}
\end{array}\right]^\top
\end{align*}
their outer product, denoted $\mathbf{a} \otimes \mathbf{b}$, is defined as the $m \times n$ matrix $\mathbf{C}$ obtained by multiplying each element of $\mathbf{a}$ by each element of $\mathbf{b}$:
\begin{align*}
\mathbf{a} \otimes \mathbf{b}=\mathbf{C}=\left[\begin{array}{cccc}
a_{1} b_{1} & a_{1} b_{2} & \ldots & a_{1} b_{n} \\
a_{2} b_{1} & a_{2} b_{2} & \ldots & a_{2} b_{n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m} b_{1} & a_{m} b_{2} & \ldots & a_{m} b_{n}
\end{array}\right]
\end{align*}
Or, in index notation,
$$ C_{ij} = a_i b_j $$

In [ ]:
a = torch.arange(3)
b = torch.arange(3,7)
c = torch.einsum('i, j -> ij', (a, b))
print_arr(a, b, c)

In [ ]:
# Using the standard PyTorch API
torch.outer(a, b)

In [ ]:
# Using broadcasting black magic
a[:, None] * b[None, :]

###### 📖 **Batch matrix multiplication**

$$ c_{bij} = \sum_k a_{bik} b_{bkj} $$

In [ ]:
a = torch.randn(2,2,5)
b = torch.randn(2,5,3)
c = torch.einsum('bik,bkj->bij', [a, b])
print_arr(a, b, c)

###### **EXERCISE**
> Implement:
> - Matrix transpose with einsum, in particular assume you have a batch of images of shape $(B, C, H, W)$ and you want to turn it into having shape $(B, H, W, C)$
> - Quadratic form with einsum  ($y = v^TMv$)

#### 📖 Singleton dimensions

 In deep learning it is very common to **add or remove dimensions of size $1$** in a tensor. As we mentioned, this is called **unsqueezing** and **squeezing**, and it occurs often during batch processing, manipulating feature maps, making network layers compatible, and in several other occasions.

 It is possible to perform these operations in different ways, feel free to use
 whatever is more comfortable to you! Again, **prefer readability to cryptic one-liners** for the sanity of a hypothetical unknown reader or your future self.

In the example below, we transform a rank-1 tensor into a rank-2 "column", and back to a rank-1:

In [ ]:
# Define a rank-1 tensor we will use
x = torch.arange(6)
print_arr(x)

Transform **`x` into a column tensor** in four different ways.

Remember that the shape of a column tensor is in the form: `(rows, 1)`

In [ ]:
# 1)
# Use the `reshape` or `view` functions

y1 = x.reshape(-1, 1)
y2 = x.view(-1, 1)

print_arr(y1, y2)

In [ ]:
# 2)
# Use the specific `unsqueeze` function to unsqueeze a dimension

y3 = x.unsqueeze(dim=-1)
y4 = x.unsqueeze(dim=1)

print_arr(y3, y4)

In [ ]:
# 3)
# Explicitly index a non-existing dimension with `None`

y5 = x[:, None]

print_arr(y5)

In [ ]:
# 4)
# Same as before, but do not assume a rank-2 tensor and index the last one.
# This approach is useful to write functions that work both for
# batched or non-batched data

y6 = x[..., None]

print_arr(y5)

In [ ]:
# Now we go back to a rank-1 tensor

x1 = y1.reshape(-1)
x2 = y2.view(-1)          # Explicity enforce to get a view of the tensors, without copying data
x3 = y3.squeeze(dim=-1)
x4 = y4.squeeze(dim=1)
x5 = y5[:, 0]             # Manually collapse the dimension with an integer indexing
x6 = y6[..., 0]

print_arr(x1, x2, x3, x4, x5, x6)

> **NOTE**
>
> indexing with `...` means  **keeping all the other dimensions the same**.
> Keep in mind that `...` is just a Python singleton object (just as `None`).
> Its type is Ellipsis:


In [ ]:
...

In [ ]:
x = torch.rand(3,3,3)
x[:, :, 0]

In [ ]:
x[..., 0]

### Tensor types
Pay attention to the tensor types!
Several methods are available to convert tensors to different types:

In [ ]:
a = torch.rand(3, 3) + 0.5

In [ ]:
a.int()

In [ ]:
a.long()

In [ ]:
a.float()

In [ ]:
a.double()

In [ ]:
a.bool()

In [ ]:
a.to(torch.double)

In [ ]:
a.to(torch.uint8)

In [ ]:
a.bool().int()

**Pro tip:** Do not try to memorize all the PyTorch API!

> Learn to understand what operation should already exist and search for it, when you need it. If it is something common, and it usually is, chances are it already exists.

Google, StackOverflow and the docs are your friends!

### 📖 Einops

If you liked the `einsum` operation, have fun with the [einops](https://github.com/arogozhnikov/einops) package! 🚀

It is a third-party library, compatible with most frameworks, that brings superpowers to `einsum`. We will not use the `einops` library in the tutorials, however, feel free to read the [docs](https://github.com/arogozhnikov/einops) and use it.

![](http://arogozhnikov.github.io/images/einops/einops_video.gif)


### Final exercises

These final exercises are designed to showcase the elegant solutions of einsum.

Feel free to also write down solutions that do _not_ use einsum, but rather with standard tensor manipulation!


#### **EXERCISE 1**
>
> You are given $b$ images with size $w \times h$. Each pixel in each image has three color channels, `(r, g, b)`. These images are organized in a tensor $X \in \mathbb{R}^{w \times b \times c \times h}$.
>
> You want to apply a linear trasformation to the color channel of each single image. In particular, you want to :
> - **Convert each image into a grey scale image**.
> - **Afterthat, transpose the images** to swap the height and width.
>
> The linear traformation that converts from `(r, g, b)` to grey scale is simply a linear combination of `r`, `g` and `b`. It can be encoded in the following 1-rank tensor $y \in \mathbb{R}^3$:

In [ ]:
y = torch.tensor([0.2989, 0.5870, 0.1140], dtype=torch.float)


> At the end, you want to obtain a tensor $Z \in \mathbb{R}^{b \times w \times h}$.
>
> Write the PyTorch code that performs this operation.

In [ ]:
# Create the input tensors for the exercise
# Execute and ignore this cell

from skimage import io
from skimage.transform import resize

size = 100

image1 = io.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Earth_Eastern_Hemisphere.jpg/260px-Earth_Eastern_Hemisphere.jpg')
image1 = torch.from_numpy(resize(image1, (size, size), anti_aliasing=True)).float()  # Covert  to float type
image1 = image1[..., :3]  # remove alpha channel

image2 = io.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/b/b4/The_Sun_by_the_Atmospheric_Imaging_Assembly_of_NASA%27s_Solar_Dynamics_Observatory_-_20100819.jpg/628px-The_Sun_by_the_Atmospheric_Imaging_Assembly_of_NASA%27s_Solar_Dynamics_Observatory_-_20100819.jpg')
image2 = torch.from_numpy(resize(image2, (size, size), anti_aliasing=True)).float()
image2 = image2[..., :3]  # remove alpha channel

image3 = io.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/Wikipedia-logo-v2.svg/1920px-Wikipedia-logo-v2.svg.png')
image3 = torch.from_numpy(resize(image3, (size, size), anti_aliasing=True)).float()
image3 = image3[..., :3]  # remove alpha channel

source_images = torch.stack((image1, image2, image3), dim=0)
images = torch.einsum('bwhc -> wbch', source_images)

In [ ]:
# Plot source images
plot_row_images(source_images)

In [ ]:
# ✏️ your code here

In [ ]:
# @title 👀 Solution


# Grey-fy all images together, using the `images` tensor
gray_images = torch.einsum('wbch, c -> bwh', (images, y))

# What if you want to transpose the images?
gray_images_tr = torch.einsum('wbch, c -> bhw', (images, y))

In [ ]:
# Plot the gray images
plot_row_images(gray_images)

In [ ]:
# Plot the gray transposed images
plot_row_images(gray_images_tr)

#### **EXERCISE 2**
>
> Given $k$ points organized in a tensor $X \in \mathbb{R}^{k \times 2}$ apply a reflection along the $y$ axis as a linear transformation.


In [ ]:
# Define some points in R^2
x = torch.arange(100, dtype=torch.float)
y = x ** 2

# Define some points in R^2
data = torch.stack((x, y), dim=0).t()

In [ ]:
px.scatter(x = data[:, 0].numpy(), y = data[:, 1].numpy())

In [ ]:
# ✏️ your code here

In [ ]:
# @title 👀 Solution


# Define a matrix that encodes a linear transformation
S = torch.tensor([[-1, 0],
                  [ 0, 1]], dtype=torch.float)

# Apply the linear transformation: the order is important
new_data = torch.einsum('nk, kd -> nd', (data, S))

# Double check yourself:
# The linear transformation correctly maps the basis vectors!
S @ torch.tensor([[0],
                  [1]], dtype=torch.float)
S @ torch.tensor([[1],
                  [0]], dtype=torch.float)

# Check if at least the shape is correct
new_data.shape

In [ ]:
# Plot the new points
px.scatter(x = new_data[:, 0].numpy(), y = new_data[:, 1].numpy())

#### **EXERCISE 3**
>
>  You are given $b$ images with size $w \times h$. Each pixel in each image has `(r, g, b)` channels. These images are organized in a tensor $X \in \mathbb{R}^{w \times b \times c \times h}$, i.e. the same tensor as in the exercise 1.
>
> You want to swap the `red` color with the `blue` color, and decrease the intensity of the `green` by half.
>
> Perform the transformation on all the images simultaneously.

In [ ]:
images.shape

In [ ]:
# ✏️ your code her

In [ ]:
# @title 👀 Solution


# Define the linear transformation to swap the blue and red colors
# and half the green
S = torch.tensor([[ 0, 0, 1],
                  [ 0, .5, 0],
                  [ 1, 0, 0]], dtype=torch.float)

# Apply the linear transformation to the color channel!
rb_images = torch.einsum('wbch, dc -> wbdh', (images, S))

# Let's try to read into the einsum operation above.
#
# For each pixel p=(r,g,b) in each image of the images tensor, we want to apply
# the transformation S @ p. In einsum notation, this would be a transformation
# represented as 'dc, c -> d' or equivalently 'c, dc -> d' if we swap the order
# of the two arguments.
#
# Now, our pixels are stored in a 4d tensor with dimensions wbch. All we want
# einsum to do, is loop over the dimensions w, b, and h, extract a pixel at each
# iteration, and apply the 'c, dc -> d' transformation to it.
#
# So here's how all the above would read in einsum notation:
#   'wbch, dc -> wbdh'
# Basically the w, b, h dimensions stay the same (we're just looping over them),
# while the c dimension gets transformed to d by application of S.

# Finally permute the dimensions so that we can use plot_row_images().
rb_images = rb_images.permute(1, 0, 3, 2)

In [ ]:
# @title 👀 Alternative solution


S = torch.tensor([[ 0, 0, 1],
                  [ 0, .5, 0],
                  [ 1, 0, 0]], dtype=torch.float)

# Instead of explicitly permuting the dimensions of the final result
# to make it work with plot_row_images(), we can directly ask einsum
# to apply a permutation for us!
#
# We just change the output dimensions 'wbdh' to 'bwhd':
rb_images = torch.einsum('wbch, dc -> bwhd', (images, S))

In [ ]:
plot_row_images(rb_images)

#### 📖 **EXERCISE 4**
>
>  You are given $b$ images with size $w \times h$. Each pixel in each image has `(r, g, b)` colors. These images are organized in a tensor $X \in \mathbb{R}^{w \times b \times c \times h}$, i.e. the same tensor as exercise 1 and 3.
>
> You want to **convert each image into a 3D point cloud**:
> - the `(x, y)` coordinates of each point in the point cloud are the **indices** of the pixels in the original image
> - the `z` coordinate of each point in the point cloud is the $L_2$ norm of the color of the corresponding pixel, multiplied by $10$
>
> *Hint*: you may need some other PyTorch function, search the docs!

In [ ]:
# @title 👀 Solution


# Just normalize the tensor into the common form [batch, width, height, colors]
imgs = torch.einsum('wbch -> bwhc', images)
imgs.shape

# The x, y coordinate of the point cloud are all the possible pairs of indices (i, j)
row_indices = torch.arange(imgs.shape[1], dtype=torch.float)
col_indices = torch.arange(imgs.shape[2], dtype=torch.float)
xy = torch.cartesian_prod(row_indices , col_indices)

# Compute the L2 norm for each pixel in each image
depth = imgs.norm(p=2, dim = -1)
# depth = torch.einsum('bwhc, bwhc -> bwh', (imgs, imgs)) ** (1/2)

# For every pair (i, j), retrieve the L2 norm of that pixel
z = depth[:, xy[:, 0].long(), xy[:, 1].long()] * 10

# Adjust the dimensions, repeat and concatenate accordingly
xy = xy.repeat(imgs.shape[0], 1, 1)  # x,y coordinates are constant for the three images
clouds = torch.cat((xy, z[..., None] ), dim= 2)

# Three images, 10000 points, each point with coordinates x,y,z in 3D
clouds.shape

In [ ]:
# Utility function
# Execute and ignore this cell

from typing import Union

def plot_3d_point_cloud(cloud: Union[torch.Tensor, np.ndarray]) -> None:
  """ Plot a single 3D point cloud

  :param cloud: tensor with shape [number of points, coordinates]
  """
  import pandas as pd
  df = pd.DataFrame(np.asarray(cloud), columns=['x', 'y', 'z'])
  fig = px.scatter_3d(df, x=df.x, y=df.y, z=df.z, color=df.z, opacity=1, range_z=[0, 30])
  fig.update_layout({'scene_aspectmode': 'data', 'scene_camera':  dict(
          up=dict(x=0., y=0., z=0.),
          eye=dict(x=0., y=0., z=3.)
      )})
  fig.update_traces(marker=dict(size=3,),
                    selector=dict(mode='markers'))
  _ = fig.show()

In [ ]:
plot_3d_point_cloud(clouds[0, ...])

In [ ]:
plot_3d_point_cloud(clouds[1, ...])

In [ ]:
plot_3d_point_cloud(clouds[2, ...])